In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import scipy.spatial
import nltk
import tensorflow as tf

In [27]:
with open("/data/glove.6B/glove.6B.100d.txt") as f:
    lines = f.readlines()
lines[:1], len(lines)

(['the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062\n'],
 400000)

In [28]:
emb_size = 100
glove = {}
for line in lines:
    parts = line.split()
    if len(parts) == emb_size + 1:
        glove[parts[0]] = np.asarray(parts[1:], dtype=np.float32)
    else:
        print("Malformed", line)
len(glove)

400000

In [3]:
def load_imdb(path):
    from bs4 import BeautifulSoup
    import re, json
    import pandas as pd
    
    def preprocess(text):
        text = BeautifulSoup(text.lower(), "html5lib").text #removed html tags
        text = re.sub(r"[\W]+", " ", text)
        return text
    
    with open(path, "r", encoding="utf8") as f:
        comments = pd.DataFrame.from_dict([json.loads(line) for line in f])
        
        
        comments["content"] = comments["content"].apply(preprocess)
        return comments
        
comments = load_imdb("/data/imdb-comments.json")
comments.head()

,content,label,name,sentiment
0,i went and saw this movie last night after bei...,test,0_10.txt,pos
1,actor turned director bill paxton follows up h...,test,10000_7.txt,pos
2,as a recreational golfer with some knowledge o...,test,10001_9.txt,pos
3,i saw this film in a sneak preview and it is d...,test,10002_8.txt,pos
4,bill paxton has taken the true story of the 19...,test,10003_8.txt,pos


In [30]:
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [31]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(comments["content"])
encoded_docs = tokenizer.texts_to_sequences(comments["content"])
vocab_size = len(tokenizer.word_index) + 1
vocab_size

103891

In [9]:
type(encoded_docs)

list

In [10]:
str(encoded_docs[0])

'[9, 416, 2, 210, 10, 15, 238, 311, 100, 109, 28203, 5, 33, 3, 173, 352, 4, 1758, 9, 232, 975, 11, 9, 13, 5735, 5, 65, 7, 84, 37, 47, 9, 672, 4, 9942, 9315, 24, 13, 62, 477, 5, 78, 201, 9, 13, 356, 9315, 254, 1, 104, 4, 3373, 15901, 53, 69, 2, 1621, 7132, 254, 1155, 8399, 17, 139, 11306, 1, 2016, 4, 3, 49, 15, 6, 11, 7, 50, 2970, 17, 258, 1334, 10, 28, 117, 619, 11, 1, 442, 764, 61, 13, 2986, 43, 13, 3232, 33, 2142, 303, 1, 88, 304, 4, 1, 15, 2, 71, 1649, 5, 1716, 303, 1, 338, 304, 136, 11778, 1, 764, 9, 23, 62, 210, 107, 362, 8, 1716, 18, 107, 364, 2061, 339, 14, 69, 264, 2816, 23, 5, 276, 246, 65, 92, 2488, 10, 15, 13, 80, 2, 9, 1466, 11, 20, 141, 65, 7, 161, 20, 1671]'

In [11]:
comments.content[0]

'i went and saw this movie last night after being coaxed to by a few friends of mine i ll admit that i was reluctant to see it because from what i knew of ashton kutcher he was only able to do comedy i was wrong kutcher played the character of jake fischer very well and kevin costner played ben randall with such professionalism the sign of a good movie is that it can toy with our emotions this one did exactly that the entire theater which was sold out was overcome by laughter during the first half of the movie and were moved to tears during the second half while exiting the theater i not only saw many women in tears but many full grown men as well trying desperately not to let anyone see them crying this movie was great and i suggest that you go see it before you judge '

In [32]:
embeddings_maxtrix = np.zeros((min(vocab_size, len(glove)), emb_size))
for word, index in tokenizer.word_index.items():
    if word in glove and index < len(glove):
        embeddings_maxtrix[index] = glove[word]
embeddings_maxtrix.shape

(103891, 100)

In [33]:
max_len = int(np.percentile([len(indices) for indices in encoded_docs], 99))
max_len

920

In [40]:
padded_docs = pad_sequences(encoded_docs, maxlen=max_len, padding='pre')
padded_docs.shape

(50000, 920)

In [41]:
print(list(padded_docs[0]))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [42]:
labels = np.where(comments.sentiment == "pos", 1, 0)
print(comments.sentiment.value_counts())

neg    25000
pos    25000
Name: sentiment, dtype: int64


In [43]:
is_training = comments.label == "train"

In [44]:
def pickleit(obj, filename):
    import pickle
    with open(filename, "wb") as f:
        pickle.dump(obj, f)
pickleit((embeddings_maxtrix, padded_docs, is_training, labels), "/tmp/imdb.pickle")

In [45]:
import tensorflow as tf
import keras
import numpy as np

def unpickle(filename):
    import pickle
    with open(filename, "rb") as f:
        return pickle.load(f)

embeddings_maxtrix, padded_docs, is_training, labels = unpickle("/tmp/imdb.pickle")

In [52]:
from keras.layers import Embedding, Dense, Flatten, Dropout, Convolution1D, MaxPooling1D

np.random.seed(1)
tf.set_random_seed(1)

e = Embedding(embeddings_maxtrix.shape[0]
              , embeddings_maxtrix.shape[1]
              , weights=[embeddings_maxtrix]
              , input_length=max_len
              , trainable=True)

model = keras.Sequential()
model.add(e)
model.add(Convolution1D(32, 10, activation = "elu"))
model.add(MaxPooling1D(3))
model.add(Dropout(0.5))
model.add(Convolution1D(64, 5, activation = "elu"))
model.add(Dropout(0.5))
model.add(MaxPooling1D(3))
model.add(Convolution1D(128, 3, activation="elu"))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

adam = keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

model.fit(padded_docs[is_training], labels[is_training]
          , validation_data = (padded_docs[~is_training], labels[~is_training])
          , batch_size=32
          , epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 920, 100)          10389100  
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 911, 32)           32032     
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 303, 32)           0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 303, 32)           0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 299, 64)           10304     
_________________________________________________________________
dropout_20 (Dropout)         (None, 299, 64)           0         
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 99, 64)            0         
__________